### Load modules

In [20]:
import numpy as np
import cv2
import sys
import os
import torch
import numpy as np

from pathlib import Path

import decord
from decord import VideoReader
from decord import cpu, gpu
decord.bridge.set_bridge('torch')

In [21]:
print(os.getcwd())
VIDEO_DIR = Path(os.getcwd()).parent / "example"

/e/workplace/hackathons/27-09-tagging-video/archive/MA-LMM-main/mixture


### Frame mixture function

### Load video

In [22]:
def video_mixer(
        vr : VideoReader, start_time=0, end_time=None,
        threshold=120, frame_limit=100, step_frames=5
        ):
    """
    Смешивает кадры видео, используя заданный порог различия и лимит кадров.

    Аргументы:
        vr (VideoReader): Объект чтения видео.
        start_time (int, optional): Начальное время в секундах. Defaults to 0.
        end_time (int, optional): Конечное время в секундах. Defaults to None.
        threshold (int, optional): Порог различия между кадрами. Defaults to 120.
        frame_limit (int, optional): Лимит кадров для смешивания. Defaults to 100.
        step_frames (int, optional): Шаг кадров для чтения видео. Defaults to 5.

    Yield:
        tuple (numpy.ndarray, list): Смешанный кадр и средний идентификатор смешанных кадров.
    """
    if end_time is None:
        end_time = len(vr)

    frame_id_list = range(start_time, end_time, step_frames)
    mixed_frame, frame_iter = None, 0
    ids_mixes_list = []
    id_begin, id_end = None, None
    batch = vr.get_batch(frame_id_list)
    for i, frame in enumerate(batch):
        frame = frame.numpy()
        if mixed_frame is None:
            mixed_frame, frame_iter = frame, 1
            id_begin = i * step_frames
            continue
        if frame_iter >= frame_limit:
            id_end = i * step_frames
            mean_id_mix = (id_begin + id_end) // 2

            yield mixed_frame, mean_id_mix
            mixed_frame, frame_iter = frame, 1
            id_begin = (i+1) * step_frames
            continue

        frame_iter += 1
        # diff = abs(frame.numpy() - mixed_frame.numpy())
        diff = abs(frame - mixed_frame)

        if diff.mean() > threshold:
            # yield torch.tensor(mixed_frame)
            id_end = i * step_frames
            mean_id_mix = (id_begin + id_end) // 2
            yield mixed_frame, mean_id_mix
            mixed_frame, frame_iter = frame, 1
            id_begin = (i+1) * step_frames
            continue

        # mixed_frame = cv2.addWeighted(mixed_frame.numpy(), 0.5, frame.numpy(), 0.5, 0)
        mixed_frame = cv2.addWeighted(mixed_frame, 0.5, frame, 0.5, 0)
        # mixed_frame = torch.tensor(mixed_frame)
    id_end = len(batch) * step_frames
    mean_id_mix = (id_begin + id_end) // 2
    yield mixed_frame, mean_id_mix

### Load modules

In [23]:
def to_tensor(mix_list, start_time, end_time, fps) -> torch.Tensor:
    return mix_list.permute(3, 0, 1, 2).to(torch.float32)

In [24]:
file_path =  str(VIDEO_DIR / "video.mp4")
vr = VideoReader(file_path, ctx=cpu(0), num_threads=4)
total_frames = len(vr)
fps = int(vr.get_avg_fps())
duration = total_frames / fps
print("video_duration: {:.1f}, fps: {:.1f}".format(duration, fps))
print(f"frames: {total_frames}")

video_duration: 70.3, fps: 24.0
frames: 1687


In [25]:
th = 255/2
sec = 10
k = 3
gen = video_mixer(vr,
                threshold=th,
                frame_limit=fps*sec,
                step_frames=k,
                )
mix_lst, mean_id_mixes = zip(*gen)

# k = 3
# mix_lst = list(video_mixer(vr,
#                            threshold=255/2,
#                            frame_limit=k*int(fps),
#                            step_frames=4,
#                            ))
len(mix_lst)

24

In [26]:
mean_id_mixes

(55,
 115,
 121,
 231,
 529,
 723,
 727,
 765,
 801,
 804,
 807,
 823,
 862,
 888,
 891,
 894,
 897,
 1003,
 1117,
 1246,
 1423,
 1524,
 1570,
 1632)

In [27]:
test_fps = 1.0
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

width, height = mix_lst[0].shape[1], mix_lst[0].shape[0]

# Create a VideoWriter object
video_writer = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

# Iterate through the frames and write them to the video
for frame in mix_lst:
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

    video_writer.write(frame)

# Release the VideoWriter object
video_writer.release()

In [28]:
# torch.Size([3, frames_size=20, 360, 640])
# video.shape